In [1]:
import pandas as pd
import pathlib
import polyflexmd.data_analysis.data.constants as data_constants
import polyflexmd.data_analysis.transform.transform as transform
import polyflexmd.data_analysis.theory.kremer_grest as kremer_grest
import numpy as np
import dask.dataframe
import dask.distributed

import copy

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
PATH_DF_TRAJECTORY_PROCESSED = pathlib.Path("/home/egor/Projects/polyflexmd/data/trajectories_test.csv")

In [4]:
client = dask.distributed.Client(n_workers=8, processes=True)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 15.51 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33229,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 15.51 GiB
Comm: tcp://127.0.0.1:33117,Total threads: 1
Dashboard: http://127.0.0.1:33855/status,Memory: 1.94 GiB
Nanny: tcp://127.0.0.1:38859,


In [5]:
traj_column_types = copy.deepcopy(data_constants.RAW_TRAJECTORY_DF_COLUMN_TYPES)
traj_column_types.pop("ix")
traj_column_types.pop("iy")
traj_column_types.pop("iz")
traj_column_types["kappa"] = "category"

df_trajectories = dask.dataframe.read_csv(
        PATH_DF_TRAJECTORY_PROCESSED,
        dtype=traj_column_types
)

time_steps_per_partition = 30000

divisions = df_trajectories["t"].loc[
    df_trajectories["t"] % time_steps_per_partition == 0
    ].unique().compute().sort_values().tolist()

df_trajectories = df_trajectories.set_index("t", divisions=divisions)

df_trajectories.persist()

df_trajectories

,id,type,x,y,z,molecule-ID,kappa
npartitions=2832,,,,,,,
450030000,uint16,uint8,float32,float32,float32,int64,category[unknown]
450060000,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
534960000,...,...,...,...,...,...,...
534990000,...,...,...,...,...,...,...


In [6]:
df_trajectories.head(5)

,id,type,x,y,z,molecule-ID,kappa
t,,,,,,,
450030000,1,1,0.000000,0.000000,0.000000,1,190.2
450030000,2,1,-0.336178,0.186169,0.890632,1,190.2
450030000,3,2,-0.635311,0.332767,1.849650,1,190.2
450030000,4,2,-0.900711,0.595613,2.730620,1,190.2
450030000,5,2,-1.188560,0.787639,3.609750,1,190.2


In [7]:
N_beads = 64
kappa = 190.2
l_b = .97

In [8]:
l_k_theory = kremer_grest.bare_kuhn_length(kappa, l_b)
l_k_theory

368.018

In [9]:
l_K_estimate = transform.estimate_kuhn_length_df(
    df_trajectory=df_trajectories,
    group_by_params=["kappa"],
    N_beads=N_beads,
    l_b=l_b
).compute()
l_K_estimate

,,l_K,d_l_K
t,kappa,,
450030000,190.2,428.037989,2.493113
450040000,190.2,389.221405,1.571433
450050000,190.2,371.061552,1.378175
450060000,190.2,362.321018,1.278343
450070000,190.2,344.221371,1.648810
...,...,...,...
534960000,190.2,361.655764,1.652846
534970000,190.2,376.939321,1.604062
534980000,190.2,354.721865,1.545850


In [10]:
l_K_estimate['l_K'].mean()

373.61305845426693

In [11]:
l_K_estimate['l_K'].std()

36.86202451568973